In [1]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8400\2936652086.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


conclusion: 

Feature Selection: CCI, ADX, Chaikin A/D
Model Selection: best using SVC with poly kernel
Hyperparameter Tuning: d = 2 c = 0.3

Report

1. SVM -> stock price -> goblok tebak all up or down
2. Data collection -> jelasin features dari alphavantage + feature buatan sendiri (short SMA / long SMA ratio)
3. Model Selection -> analyze correlation  bilkang not linearly separable ...
4. Feature Selection
5. Hyperparameter tuning

stevan: 
- feature buatan sendiri (short SMA / long SMA ratio)
- training data selection
- improve acc to >70%

In [2]:
'''
CONST
'''
FILEPATH_STOCK_PRICE = "./tr_eikon_eod_data.csv"
STOCK_NAMES = ['AAPL.O', 'MSFT.O', 'AMZN.O', 'AMZN.O', 'AMZN.O']

In [3]:
''' 
UTILS
'''
def sequential_train_test_split(data, ratio = 0.8):
    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df[split_index:]

def calculate_return(prediction, data) :
    data = data
    prediction = pd.DataFrame(prediction)
    temp = pd.DataFrame(data)
    temp['log_return'] = np.log(temp / temp.shift(1))
    temp['returne'] = prediction.shift(1) * temp['log_return'].values.reshape(-1, 1)
    temp.dropna(inplace = True) 

    return np.exp(temp[['log_return', 'returne']].sum())


In [4]:
'''
BENCHMARK
'''

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    return (prediction == test).sum() / len(prediction)


In [5]:
''' 
MODEL
'''

def predict_position(train, test):
    # Train Hidden Markov Model
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(train.reshape(-1, 1))

    # Predict hidden states for test data
    hidden_states = model.predict(test.reshape(-1, 1))

    # Define a function to map hidden states to rise or fall
    def map_to_rise_or_fall(state):
        return 1 if state == 0 else -1

    # Map hidden states to rise or fall
    predicted_rise_fall = np.array(list(map(map_to_rise_or_fall, hidden_states)))

    return predicted_rise_fall

In [8]:
apple_data = pd.read_csv('amazon_output.csv', index_col = 0, parse_dates = True).dropna()
print(apple_data)
apple_data['SMA_EMA_ratio'] = apple_data['SMA'] / apple_data['EMA']
SMA_short = 7
SMA_long = 20
apple_data['SMA_Short'] = apple_data['AMZN.O'].rolling(SMA_short).mean()
apple_data['SMA_Long'] = apple_data['AMZN.O'].rolling(SMA_long).mean()
apple_data['SMA_Long_Short'] = apple_data['SMA_Short'] / apple_data['SMA_Long']
apple_data['data'] = apple_data['AMZN.O']
apple_data['direction'] = np.where(apple_data['AMZN.O'] <= apple_data['AMZN.O'].shift(-1), 1, -1)

apple_data

              AMZN.O    SlowK    SlowD      RSI      ADX       CCI  \
Date                                                                 
2010-04-01   133.900  56.8287  73.2678  66.1726  35.5107    8.9870   
2010-06-01   132.250  75.6546  78.7052  45.0015  45.8330   12.7304   
2010-07-01   130.000  22.3903  20.1191  21.6477  46.2261 -114.8337   
2010-11-01   130.308  21.3492  40.7736  55.0986  27.7141  -48.5113   
2010-12-01   127.350  65.8706  79.3297  67.8835  31.8447   68.1857   
...              ...      ...      ...      ...      ...       ...   
2018-06-25  1663.150  28.7844  51.2296  53.7404  46.1234 -197.7752   
2018-06-26  1691.090  25.7092  34.9854  44.5310  43.3708 -120.6630   
2018-06-27  1660.510  21.6400  25.3779  54.7483  40.0760 -113.1170   
2018-06-28  1701.450  35.8428  27.7307  45.4836  37.1107  -58.4410   
2018-06-29  1699.800  46.1879  34.5569  55.8579  33.7712   -7.2918   

            Aroon Down  Aroon Up           OBV   Chaikin A/D      SMA      EMA  
Date    

,AMZN.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,133.900,56.8287,73.2678,66.1726,35.5107,8.9870,57.1429,85.7143,1.140431e+10,2.070208e+10,6.6543,6.6815,0.995929,NaN,NaN,NaN,133.900,-1
2010-06-01,132.250,75.6546,78.7052,45.0015,45.8330,12.7304,57.1429,14.2857,1.096472e+10,2.068413e+10,6.1918,6.2536,0.990118,NaN,NaN,NaN,132.250,-1
2010-07-01,130.000,22.3903,20.1191,21.6477,46.2261,-114.8337,85.7143,21.4286,1.030262e+10,2.043025e+10,5.9468,5.8418,1.017974,NaN,NaN,NaN,130.000,1
2010-11-01,130.308,21.3492,40.7736,55.0986,27.7141,-48.5113,0.0000,64.2857,1.183112e+10,2.405573e+10,8.2560,8.2434,1.001528,NaN,NaN,NaN,130.308,-1
2010-12-01,127.350,65.8706,79.3297,67.8835,31.8447,68.1857,28.5714,85.7143,1.243995e+10,2.412455e+10,8.5062,8.6501,0.983364,NaN,NaN,NaN,127.350,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,1663.150,28.7844,51.2296,53.7404,46.1234,-197.7752,100.0000,85.7143,2.248574e+10,3.437684e+10,85.9311,85.7731,1.001842,1719.094286,1683.2580,1.021290,1663.150,1
2018-06-26,1691.090,25.7092,34.9854,44.5310,43.3708,-120.6630,92.8571,78.5714,2.257347e+10,3.441648e+10,85.8280,85.3813,1.005232,1715.540000,1687.6590,1.016521,1691.090,-1
2018-06-27,1660.510,21.6400,25.3779,54.7483,40.0760,-113.1170,85.7143,71.4286,2.247603e+10,3.432094e+10,85.8545,85.3857,1.005490,1706.500000,1690.1770,1.009658,1660.510,1


In [9]:
scaler = MinMaxScaler()

# Normalize the selected columns
normalized_columns = scaler.fit_transform(apple_data.iloc[:, :-2])

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_columns, columns=apple_data.columns[:-2])

# Display the normalized DataFrame
apple_data.iloc[:, :-2] = normalized_df
apple_data


,AMZN.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,0.015407,0.547956,0.743189,0.642768,0.470929,0.506203,0.571429,0.857143,0.125422,0.082504,0.010625,0.010581,0.445655,NaN,NaN,NaN,133.900,-1
2010-06-01,0.014402,0.745369,0.803009,0.359953,0.656789,0.511902,0.571429,0.142857,0.094772,0.081320,0.005648,0.005986,0.349605,NaN,NaN,NaN,132.250,-1
2010-07-01,0.013031,0.186826,0.158464,0.047981,0.663867,0.317693,0.857143,0.214286,0.048608,0.064585,0.003012,0.001564,0.810013,NaN,NaN,NaN,130.000,1
2010-11-01,0.013219,0.175909,0.385698,0.494836,0.330545,0.418665,0.000000,0.642857,0.155180,0.303568,0.027860,0.027355,0.538203,NaN,NaN,NaN,130.308,-1
2010-12-01,0.011417,0.642771,0.809880,0.665623,0.404920,0.596329,0.285714,0.857143,0.197630,0.308104,0.030553,0.031723,0.237986,NaN,NaN,NaN,127.350,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.947041,0.253876,0.500732,0.476692,0.662018,0.191420,1.000000,0.857143,0.898056,0.983911,0.863689,0.859967,0.543386,0.994631,0.990455,0.506771,1663.150,1
2018-06-26,0.964063,0.221629,0.322018,0.353668,0.612456,0.308819,0.928571,0.785714,0.904173,0.986523,0.862580,0.855759,0.599412,0.992430,0.993238,0.485874,1691.090,-1
2018-06-27,0.945433,0.178958,0.216319,0.490156,0.553130,0.320307,0.857143,0.714286,0.897378,0.980226,0.862865,0.855806,0.603685,0.986834,0.994830,0.455802,1660.510,1


In [10]:
# apple_data = apple_data[['SlowK', 'RSI', 'CCI', 'Aroon Down', 'OBV', 'direction']]

In [11]:
train_data, test_data = sequential_train_test_split(apple_data)
print(test_data)
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]

[[ 4.44406538e-01  6.52317985e-01  4.21603121e-01 ...  3.90079586e-01
   8.38090000e+02 -1.00000000e+00]
 [ 4.42633737e-01  8.55400942e-01  6.51868305e-01 ...  3.91299289e-01
   8.35180000e+02 -1.00000000e+00]
 [ 4.34963782e-01  7.50251146e-01  7.77093840e-01 ...  3.94158678e-01
   8.22590000e+02 -1.00000000e+00]
 ...
 [ 9.45433057e-01  1.78958296e-01  2.16319454e-01 ...  4.55802397e-01
   1.66051000e+03  1.00000000e+00]
 [ 9.70374116e-01  3.27892369e-01  2.42204209e-01 ...  4.31928489e-01
   1.70145000e+03 -1.00000000e+00]
 [ 9.69368919e-01  4.36373646e-01  3.17303884e-01 ...  4.03693133e-01
   1.69980000e+03 -1.00000000e+00]]


In [12]:
X_train

array([[5.00770651e-03, 8.40882732e-01, 8.16772503e-01, ...,
        5.19717143e-03, 2.65231084e-01, 1.16830000e+02],
       [5.73875855e-03, 8.16317649e-01, 6.25947245e-01, ...,
        4.69534548e-03, 2.84914558e-01, 1.18030000e+02],
       [6.99373123e-03, 1.42803813e-01, 2.17810181e-01, ...,
        4.31083359e-03, 2.92344218e-01, 1.20090000e+02],
       ...,
       [4.31978653e-01, 1.34644464e-01, 9.72592612e-02, ...,
        4.43306157e-01, 4.49133062e-01, 8.17690000e+02],
       [4.27488775e-01, 1.96970523e-01, 1.28883593e-01, ...,
        4.44419850e-01, 4.15678848e-01, 8.10320000e+02],
       [4.32770626e-01, 3.92174114e-01, 2.40563858e-01, ...,
        4.45645798e-01, 3.96189961e-01, 8.18990000e+02]])

In [13]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', gamma=0.7, C=1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
benchmark_accuracy(pred, y_test)

0    0.558824
dtype: float64

In [14]:
best_acc = 0
best_g = -1
best_c = -1
best_return = 0
bestgg = -1
bestcc = -1
log_return = 0  

selected_features = [4, 5, 9]
dat = X_test[:,-1]

for g in range(1, 11, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='rbf', gamma=g / 10, C=c / 10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)
        log_return = temp.values[0]
        if acc[0] > best_acc:
            best_acc = acc[0]
            best_g = g
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestgg = g
            bestcc = c

print(f"best acc: {best_acc}, g: {best_g}, c: {best_c}")
print(f"best ret: {best_return}, g: {bestgg}, c: {bestcc}")

best acc: 0.5668449197860963, g: 1, c: 1
best ret: 2.173587008619608, g: 4, c: 47


In [15]:
print(log_return)

2.0281831306900227


In [16]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]
dat = X_test[:,-1]

for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='sigmoid', C= c/10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5668449197860963, c: 1
best ret: 2.0316830692350827, c: 9


In [17]:
best_acc = 0
best_d = -1
best_c = -1
best_return = 0
bestdd = -1
bestcc = -1

selected_features = [1, 5, 11, 13]
dat = X_test[:,-1]

for d in range(1, 5, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree=d, C=c/10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_d = d
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestdd = d
            bestcc = c
    
print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")
print(f"best ret: {best_return}, d : {bestdd}, c: {bestcc}")

best acc: 0.5668449197860963, d: 1, c: 1
best ret: 2.0782692785726593, d : 3, c: 7


In [18]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]

# for d in range(1, 5, 1):
for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='poly', degree=2, C = c / 10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5668449197860963, c: 1
best ret: 2.0281831306900227, c: 1


In [19]:
# Feature Engineering

# Feature Selection

In [20]:
X = apple_data.dropna().iloc[:, :-1]
y = apple_data.dropna().iloc[:, -1]

In [21]:
X

,AMZN.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data
Date,,,,,,,,,,,,,,,,,
2010-08-02,0.005008,0.840883,0.816773,0.462332,0.488492,0.625155,0.571429,0.000000,0.017460,0.172783,0.002090,0.001870,0.487037,0.005337,0.005197,0.265231,116.83
2010-09-02,0.005739,0.816318,0.625947,0.608273,0.119056,0.813141,0.714286,1.000000,0.095017,0.222883,0.007046,0.006935,0.460036,0.005208,0.004695,0.284915,118.03
2010-11-02,0.006994,0.142804,0.217810,0.474477,0.300651,0.449268,0.000000,0.571429,0.161120,0.303394,0.028025,0.027245,0.589425,0.004973,0.004311,0.292344,120.09
2010-12-02,0.006732,0.456129,0.649988,0.590732,0.400390,0.558388,0.214286,0.785714,0.189908,0.311599,0.031576,0.032100,0.353277,0.004410,0.003984,0.277885,119.66
2010-02-16,0.005434,0.630187,0.661133,0.374768,0.173179,0.562233,0.285714,0.214286,0.047800,0.024714,0.002630,0.002960,0.345495,0.003713,0.003580,0.259807,117.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.947041,0.253876,0.500732,0.476692,0.662018,0.191420,1.000000,0.857143,0.898056,0.983911,0.863689,0.859967,0.543386,0.994631,0.990455,0.506771,1663.15
2018-06-26,0.964063,0.221629,0.322018,0.353668,0.612456,0.308819,0.928571,0.785714,0.904173,0.986523,0.862580,0.855759,0.599412,0.992430,0.993238,0.485874,1691.09
2018-06-27,0.945433,0.178958,0.216319,0.490156,0.553130,0.320307,0.857143,0.714286,0.897378,0.980226,0.862865,0.855806,0.603685,0.986834,0.994830,0.455802,1660.51


In [22]:
y

Date
2010-08-02    1
2010-09-02    1
2010-11-02   -1
2010-12-02   -1
2010-02-16   -1
             ..
2018-06-25    1
2018-06-26   -1
2018-06-27    1
2018-06-28   -1
2018-06-29   -1
Name: direction, Length: 1868, dtype: int32

In [23]:
best_acc = 0
best_ret = 0

dat = X_test[:,-1]
for i in range(len(apple_data.columns) - 1):
    print(X_train[:,i])
    curr_X_train = X_train[:,i].reshape(-1, 1)
    curr_X_test = X_test[:,i].reshape(-1, 1)
    model = SVC(kernel='rbf', gamma= 1.0, C= 1.0)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp =calculate_return(pred, dat).values[1]
    if temp > best_ret:
        best_ret = temp
    
    print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")
print(best_ret)

[0.00500771 0.00573876 0.00699373 ... 0.43197865 0.42748878 0.43277063]
feature 0: acc: 0.567 up: 1.0
[0.84088273 0.81631765 0.14280381 ... 0.13464446 0.19697052 0.39217411]
feature 1: acc: 0.567 up: 1.0
[0.8167725  0.62594724 0.21781018 ... 0.09725926 0.12888359 0.24056386]
feature 2: acc: 0.567 up: 1.0
[0.46233165 0.60827346 0.47447721 ... 0.40126986 0.323904   0.27410344]
feature 3: acc: 0.567 up: 1.0
[0.48849164 0.11905564 0.30065055 ... 0.36678196 0.37299214 0.36128302]
feature 4: acc: 0.567 up: 1.0
[0.6251554  0.81314062 0.44926839 ... 0.35042408 0.30130275 0.39446243]
feature 5: acc: 0.567 up: 1.0
[0.571429 0.714286 0.       ... 0.857143 1.       0.928571]
feature 6: acc: 0.567 up: 1.0
[0.       1.       0.571429 ... 0.357143 0.285714 0.214286]
feature 7: acc: 0.567 up: 1.0
[0.01746039 0.09501672 0.16112042 ... 0.67513883 0.67074353 0.6746383 ]
feature 8: acc: 0.567 up: 1.0
[0.17278291 0.2228828  0.30339357 ... 0.80940246 0.8100055  0.81338377]
feature 9: acc: 0.567 up: 1.0
[0.0

In [24]:
best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(len(apple_data.columns) - 1):
    for j in range(len(apple_data.columns) - 1):
        selected_features = [i, j]
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

best acc: 0.5668449197860963 features: [0, 0]
best ret: 2.0281831306900227 features: [0, 0]


In [25]:
print(apple_data.columns)

Index(['AMZN.O', 'SlowK', 'SlowD', 'RSI', 'ADX', 'CCI', 'Aroon Down',
       'Aroon Up', 'OBV', 'Chaikin A/D', 'SMA', 'EMA', 'SMA_EMA_ratio',
       'SMA_Short', 'SMA_Long', 'SMA_Long_Short', 'data', 'direction'],
      dtype='object')


In [32]:
import itertools

all_features = list(range(len(apple_data.columns) - 2))

best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(1, len(apple_data.columns) - 2):
    print(i)
    possible_selected_features = list(itertools.combinations(all_features, i))
    for selected_features in possible_selected_features:
        print(selected_features)
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

1
(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
2
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(2, 10)
(2, 11)
(2, 12)
(2, 13)
(2, 14)
(2, 15)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(3, 10)
(3, 11)
(3, 12)
(3, 13)
(3, 14)
(3, 15)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(4, 9)
(4, 10)
(4, 11)
(4, 12)
(4, 13)
(4, 14)
(4, 15)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(6, 7)
(6, 8)
(6, 9)
(6, 10)
(6, 11)
(6, 12)
(6, 13)
(6, 14)
(6, 15)
(7, 8)
(7, 9)
(7, 10)
(7, 11)
(7, 12)
(7, 13)
(7, 14)
(7, 15)
(8, 9)
(8, 10)
(8, 11)
(8, 12)
(8, 13)
(8, 14)
(8, 15)
(9, 10)
(9, 11)
(9, 12)
(9, 13)
(9, 14)
(9, 15)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(11, 12)
(11, 13)
(11, 14)
(11, 15)
(12, 13)
(12, 14)
(12, 15)
(13, 14

KeyboardInterrupt: 

In [34]:

print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

best acc: 0.5668449197860963 features: (0, 10)
best ret: 2.3992565838150837 features: (0, 2, 6, 8, 9, 10, 12)
